# Importação das bibliotecas

In [1]:
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import json
import sagemaker
import boto3
import io
import os
import sagemaker.amazon.common as sac 
from sagemaker import Session
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# Leitura dos dados

In [3]:
# Nome do segredo armazenado no AWS Secrets Manager
nome_segredo = "HotelDB"
nome_regiao = "us-east-1"

In [4]:
# Criando um cliente do AWS Secrets Manager
sessao = boto3.session.Session()
client = sessao.client(
    service_name="secretsmanager",
    region_name=nome_regiao,
)

In [5]:
# Recuperando o segredo
resposta_segredo = client.get_secret_value(SecretId=nome_segredo)
segredo = json.loads(resposta_segredo["SecretString"])

In [6]:
# Extraindo as credenciais do segredo
db_user = segredo["username"]
db_password = segredo["password"]
db_host = segredo["host"]

In [7]:
# Criando a conexão com o banco de dados MySQL
conexao = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}')

In [8]:
# Importando a tabela do banco de dados dentro do schema 'user'
dados = pd.read_sql('SELECT * FROM user.HotelModel', con=conexao)

# Tratamento dos dados

In [9]:
# Trocando a ordem das colunas, colocando y em primeiro
colunas = []
colunas.append('label_avg_price_per_room')
for cont in range(1,len(dados.columns[1:-1])):
    colunas.append(dados.columns[cont])

In [10]:
dados = dados[colunas]

In [11]:
# Dividindo os dados em conjunto de treino e teste
dados_treino, dados_teste = train_test_split(dados, test_size = 0.25, random_state = 0)

In [12]:
X_teste = dados_teste.iloc[:, 1:len(dados)].values

In [13]:
y_teste = dados_teste.iloc[:,0].values

In [14]:
# Convertendo os dados para CSV
dados_treino.to_csv('dados_treino.csv', header = False, index = False)
dados_teste.to_csv('dados_teste.csv', header = False, index = False)

# Configuração do SageMaker

In [15]:
Sessao = sagemaker.Session()
bucket = 'hotelbuck'
subpasta_modelo = 'modelo'
subpasta_dados = 'dados'
regra = sagemaker.get_execution_role()

In [16]:
# Armazenando os dados de treino no S3
arquivo_treino = 'dadostreino'
arquivo_teste = 'dadosteste'
s3_dados_treino = 's3://{}/{}/treino/{}'.format(bucket,subpasta_dados,arquivo_treino)
s3_dados_teste = 's3://{}/{}/teste/{}'.format(bucket,subpasta_dados,arquivo_teste)
saida = 's3://{}/{}'.format(bucket,subpasta_modelo)

In [17]:
with open('dados_treino.csv', 'rb') as arquivo:
         boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dados,'treino',arquivo_treino)).upload_fileobj(arquivo)

In [18]:
with open('dados_teste.csv', 'rb') as arquivo:
         boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dados,'teste',arquivo_teste)).upload_fileobj(arquivo)

# Treinamento do modelo

In [19]:
from sagemaker import image_uris
conteiner = image_uris.retrieve(framework = 'xgboost', region='us-east-1', version='1.7-1')

In [20]:
classificador = sagemaker.estimator.Estimator(image_uri = conteiner,
                                        role = regra,
                                        instance_count = 1,
                                        instance_type = 'ml.m5.2xlarge',
                                        output_path = saida,
                                        sagemaker_session = Sessao,
                                        use_spot_instances = True,
                                        max_run = 3600,
                                        max_wait = 3600)

In [21]:
classificador.set_hyperparameters(num_round = 1000, objective = 'multi:softmax', num_class = 16)

In [22]:
entrada = sagemaker.inputs.TrainingInput(s3_data = s3_dados_treino, content_type = 'csv', s3_data_type = 'S3Prefix')
teste = sagemaker.inputs.TrainingInput(s3_data = s3_dados_teste, content_type = 'csv', s3_data_type = 'S3Prefix')
canais = {'train': entrada, 'validation': teste }

In [23]:
classificador.fit(canais)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-08-29-16-18-24-905


2024-08-29 16:18:26 Starting - Starting the training job...
2024-08-29 16:18:41 Starting - Preparing the instances for training...
2024-08-29 16:19:24 Downloading - Downloading the training image......
2024-08-29 16:20:15 Training - Training image download completed. Training in progress....[2024-08-29 16:20:31.982 ip-10-0-230-60.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-08-29 16:20:32.004 ip-10-0-230-60.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-08-29:16:20:32:INFO] Imported framework sagemaker_xgboost_container.training
[2024-08-29:16:20:32:INFO] Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
[2024-08-29:16:20:32:INFO] No GPUs detected (normal if no gpus installed)
[2024-08-29:16:20:32:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-08-29:16:20:32:INFO] Determined 0 GPU(s) available on the instance.
[2024-08-29:16:20:32:INFO] Determined delimiter of CSV

# Deploy do modelo

In [24]:
classificador_hotel = classificador.deploy(initial_instance_count=1,instance_type='ml.m5.large') 

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-08-29-16-22-02-962
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-08-29-16-22-02-962
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-08-29-16-22-02-962


-------!

# Avaliação do modelo

In [25]:
classificador_hotel.serializer = CSVSerializer()
classificador_hotel.deserializer = JSONDeserializer()

In [39]:
resultados = classificador_hotel.predict(X_teste)

In [40]:
previsao = np.array([v['score'] for v in resultados['predictions']])
previsao

array([3., 1., 2., ..., 3., 3., 1.])

In [41]:
real = np.array(y_teste).astype(float)
real

array([3., 1., 2., ..., 3., 3., 1.])

In [42]:
accuracy_score(real,previsao)

0.8633807476017201

In [43]:
mc = confusion_matrix(real,previsao)
mc

array([[2367,  311,   54],
       [ 300, 2790,  266],
       [  22,  286, 2673]])

In [44]:
print(classification_report(real,previsao))

              precision    recall  f1-score   support

         1.0       0.88      0.87      0.87      2732
         2.0       0.82      0.83      0.83      3356
         3.0       0.89      0.90      0.89      2981

    accuracy                           0.86      9069
   macro avg       0.87      0.86      0.87      9069
weighted avg       0.86      0.86      0.86      9069

